**F1**

```py
simple.py assembly code

            BR program
x:          .BLOCK 2
program:    LDWA 3,i
            ADDA 2,i
            STWA x,d
            DECO x,d
            .END 

add_sub.py assembly code

            BR  program
value:      .BLOCK 2
_UNIV:      .WORD 42
variable:   .WORD 3
result:     .BLOCK 2
program:    DECI value, d
            LDWA _UNIV, d
            ADDA value, d
            SUBA variable, d
            SUBA 1,i
            STWA result, d
            DECO result, d
            .END
```

Global vs Local

In RBS to determine if a variable is global or local, simply check where it is declared. If it is declared outside of a function, it can be used anywhere, making it global. If it is declared within a function, it is local to that function. The global keyword can also be used within function before the variable to allow for access everywhere.

NOP1

The translator uses NOP1 instructions, as it is a unary no-operation trap. It is used to translate code into PEP9 to determine its structure. It is also used to avoid errors when jumping. As featured in our code, the BR program label at the start is a command  to indicate to branch right away to the  next BR program label. Incase the label does nothing, NOP is there to ensure the code does not crash, and still compiles. 
https://computersystemsbook.com/5th-edition/pep9/whats-new-pep-9-micro-simulator/

Visitor and Generator

The first visitor is the GlobalVariableExtraction visitor, which is used for extracting all global variables from the source code. It does this by traversing the AST and looking for nodes with a 'global' keyword. The second visitor is the TopLevelProgram visitor. This is used for generating the top level instructions for the compiled program. It does this by traversing the AST and looking for nodes with a 'main' keyword. It then generates the instructions for these nodes.

The first generator is the StaticMemoryAllocation generator. This generator is responsible for allocating memory for all global variables. It does this by using the list of global variables generated by the first visitor. Then it allocates memory for each variable and saves the address in a list. The second generator is the EntryPoint generator. This generator is responsible for generating the entry point for the compiled program,  by using the list of instructions generated by the second visitor.Then it generates the instructions for the entry point.

Improvements to the Code

The current translation code is limited in a few ways. First, it only supports translating one file at a time. This means that it is not possible to translate multiple files or libraries at once, which can be a limiting factor when trying to build larger programs. The code is not very modular, which makes it difficult to add new features or make changes to existing ones.

The current translation code is very limited in terms of software engineering. It does not support any kind of modularity or object-oriented programming, which makes it very difficult to reuse code or create new features. Additionally, the code is very difficult to read and understand, making it hard to maintain or modify. There is no clear separation of concerns between the different parts of the code. The code is not very well organized, and it is not clear what each part of the code is supposed to do. Finally, the code is not very well commented, and it is not clear what each line of code does.



**F2**


```py
; Translating _samples\2_mem_alloc\fibonnaci.py
; Branching to top level (tl) instructions
                BR tl

; SYMBOL TABLE
; Previous Variable     New Variable
; n:                    A
; a:                    B
; b:                    C
; i:                    D
; tmp:                  E

; Allocating Global (static) memory
A:              .BLOCK 2
B:              .WORD 0
C:              .WORD 1
D:              .WORD 0
E:              .BLOCK 2
; Top Level instructions
tl:             NOP1
                DECI A,d
                LDWA 0,i
                STWA B,d
                LDWA 1,i
                STWA C,d
                LDWA 0,i
                STWA D,d
test_0:         LDWA D,d
                CPWA A,d
                BRGE end_l_0
                LDWA B,d
                ADDA C,d
                STWA E,d
                LDWA C,d
                STWA B,d
                LDWA E,d
                STWA C,d
                LDWA D,d
                ADDA 1,i
                STWA D,d
                BR test_0
end_l_0:        NOP1
                DECO B,d
                .END
                ```

For constants we modify how they are created in the static memory allocation. We check if it begins with an underscore and change to an .EQUATE n. From the ast we extract the name and value .

Similarly if a .BLOCK 2 variable has a variable we change it to a word right away in the static memory allocation and assign it the value. 

For our symbol table we change the names of the variables to new names wiht a new genrator class to make sure none of the variables are longer than 8 characters.

**F3 Conditionals**

```py
gcd.py assembly code
         BR program 
a:       .BLOCK 2
b:       .BLOCK 2
program: DECI a,d
         DECI b,d
test:    LDWA a,d
         CPWA b,d
         BREQ end_w
test_if: LDWA a,d
         CPWA b,d
         BRLE else_b
if_a:    LDWA a,d
         SUBA b,d
         STWA a,d
         BR test
else_b:  LDWA b,d
         SUBA a,d
         STWA b,d
         BR test
end_w:   DECO a,d
         .END
```

To automate the translation of conditionals we must add a visit if into the TopLevelProgram.py. In RBS, there can be if, elif or else. As we can see in the ast tree, if statments are broken up into if with an orelse inside of it. If there is no orelse it is a stand alone if statement. If there is an orelse, then must check inside of the orelse body. If there is another If block witin the orelse then it must be an elif statment, if no If statement, means it is the final else in the conditional branch. Following this logic into our visit_If function within top level program, we can record instructions through access according to the condition with the load, and compare operations and record instructions to start and branch to exit the loop, with inverted conditions (similar logic to the while loop). Only the visitor class, top level program is affected, no changes to the generator, since we are visitng the If block of the AST tree. 

***F4 Function Calls***

```
; Translating _samples\4_functions\call_param.py
         BR      Program
;static memory allocation
_UNIV:   .EQUATE 42
x:       .BLOCK 2
;local variables
result:  .EQUATE 0
pValue:  .EQUATE 4
my_func: SUBSP 2, i 
         LDWA pValue, s 
         ADDA _UNIV, d
         STWA result, s   
         DECO result, s 
         ADDSP 2, i
         RET
Program: SUBSP 2, i
         DECI x, d
         LDWA x, d
         STWA 0, s
         CALL my_func
         ADDSP 2, i
         .END
```

```
; Translating _samples\4_functions\call_return.py
         BR      Program
;static memory allocation
_UNIV:   .WORD 42
x:       .BLOCK 2
;local variables
result:  .EQUATE 0
;ret add stored here
pValue:  .EQUATE 4
retVal:  .EQUATE 6
my_func: SUBSP 2, i ;push result
         LDWA pValue, s 
         ADDA _UNIV, d
         STWA result, s
         LDWA result, s   
         STWA retVal, s
         ADDSP 2, i ;pop result
         RET
Program: SUBSP 4, i ;push value and retVal
         DECI x, d
         LDWA x, d
         STWA 0, s
         CALL my_func
         LDWA 2,s
         STWA result, d
         ADDSP 4, i
         DECO result, d 
         .END
```


```
;Translating _samples\4_functions\call_void.py
;static memory allocation
_UNIV:   .WORD 42
x:       .BLOCK 2
;local variables
result:  .EQUATE 0
value:   .BLOCK 2
my_func: SUBSP 2, i ;push result and value
         DECI value, d 
         LDWA value, d 
         ADDA _UNIV, d
         STWA result, s
         DECO result, s
         ADDSP 2, i ;pop result and value
         RET
Program: CALL my_func
         .END
```

The ranking of the following files within the F4 directory with respect to their translation complexity(hardest->easiet) as follows:

1.factorial_rec
2.factorial_normal

3.fib_rec
4.fibonnaci

5.call_return
6.call_param
7.call_void

Factorial and fibonnaci are rated above the call programs because the logic is much more complex within the fib and factorial files compared to the call files. 

Within the call files, call void was rated the lowest due to having only the following properties; global and local variables. Next was call_param which had the sa,e properties as call_void but now incorporated parameters which we had to allocate using a stack. And call_return was rated the highest amongst these three because it has all these properties along with a return statmenet, which we needed to allocate more space on the stack for. 

Factorial was rated above fib since factorial uses two seperate functions increasing the difficulty to translate, and within the two respective files, the recursive file was rated ahead of the iterative file since the recursive logic is slighlty more complex when translating. 





To automate the translation of function calls we will need 2 new visitor classes and 2 generators. Local variable class to extract all the local variables in the function and a function level program to extract all the details within the function. As we can see in the ast tree, function calls are broken up into function calls with an args inside of it. If there is no args it is a stand alone function call statement. If there is an args, then must check inside of the args body. Following this logic into our visit_FunctionDef function within function level program, we can record instructions through access according to the condition with the load, and compare operations and record instructions to start and branch to exit the loop, with inverted conditions (similar logic to the while loop). The function level program appears before the top level program so a new generator local entry point was added to print the local function instructions at the correct spot. Since we have a local variable class, we also need a local status memory allocation to account for all the local variables in the fucntion, which will be printed after the global variables. 

Within in the function the call-by-value is implemented by passing the address of the variable to the function. This is done by storing the address of the variable in the stack and then loading the value of the variable into the accumulator. The function then uses the accumulator to perform the operations and stores the result back into the variable.




No stack overflow mechanism is provided, if such situation arises the pep/9 will crash or give an error. In our testing when we tried really large numbers for factorial, the output became a large negative number. Thus, we can conclude that the stack overflow mechanism is not provided.

**F5 Global and Local Arrays**

Explain in natural language how to automate the translation of global arrays and how it impacts the visit and generation. 

To change the translation of global arrays, we must change the top level program visitor class. We must add a visit for global arrays, and within the visit we must have a new way to create the load indeces in the index register (ASLX) and if an instruction uses the index register we have to change the instruction to use the index register instead of the accumulator.

For local arrays, we must store the arrays on the stack, and we must change the load and store instructions to use the stack pointer instead of the decimal value. We would need to change the visitor class for local variabe and top level program to accomodate for thee change. 


The idea behind managing unbounded data structures on a virtual machine like Pep/9 is to use a data structure called a dynamic array. A dynamic array is an array that grows in size as needed. The idea is that when the array is full, a new array of double the size is allocated and the old array is copied into the new array. This process would be repeated each time the array is filled. To manage this dynamic array, Pep/9 would need to keep track of the size of the array, the amount of space allocated, and the address of the start of the array. This would allow the virtual machine to keep track of the dynamic array and use it as needed.